# 코루틴 사용

- [1. 코루틴에 값 보내기]()
- [2. 코루틴 바깥으로 값 전달하기]()
  - [제너레이터와 코루틴의 차이점]()
- [3. 코루틴을 종료하고 예외 처리하기]()
  - [GeneratorExit 예외 처리하기]()
  - [코루틴 안에서 예외 발생시키기]()
- [4. 하위 코루틴의 반환값 가져오기]()

# 1. 코루틴에 값 보내기

---

- 코루틴은 제너레이터의 특별한 형태이다.
- 제너레이터는 yield로 값을 발생시켰지만 코루틴은 yield로 값을 받아올 수 있다.
- 코루틴에 값을 보내면서 코드를 실행할 때는 `send()` 함수를 사용한다.
- `send()` 함수가 보낸 값을 받아오려면 `(yeild)` 형식으로 yield를 괄호로 묶어준 뒤 변수를 저장한다.

In [10]:
def print_num_coroutine():
    print('RUN Coroutine')
    while True:       # 코루틴을 계속 유지하기 위해 무한 루프 사용
        print('STOP Before yield')
        x = (yield)   # 코루틴 바깥에서 값을 받아옴, yield 키워드를 괄호로 묶어줘야 함
        print(x)      # x 받아온 값 출력

co = print_num_coroutine()  # 코루틴 객체 생성

# 코루틴의 코드를 최초로 실행한 뒤 메인 루틴으로 돌아옴
next(co)    # 코루틴 안의 yield까지 코드 실행 (최초 실행)

# send()로 값을 보내고 코루틴의 yield까지의 코드가 끝나면 다시 메인 루틴으로 돌아옴
print('Ready to send(1)')
co.send(1)  # 코루틴에 숫자 1을 보냄
print('Ready to send(2)')
co.send(2)  # 코루틴에 숫자 2을 보냄
print('Ready to send(3)')
co.send(3)  # 코루틴에 숫자 3을 보냄
print('Back to Main Routine')

RUN Coroutine
STOP Before yield
Ready to send(1)
1
STOP Before yield
Ready to send(2)
2
STOP Before yield
Ready to send(3)
3
STOP Before yield
Back to Main Routine


# 2. 코루틴 바깥으로 값 전달하기

---

- `(yield 변수)` 형식으로 yield에 변수를 지정한 뒤 괄호로 묶어주면 값을 받아오면서 바깥으로 값을 전달한다

In [8]:
def sum_coroutine():
    print('RUN Coroutine')
    total = 0               # 코루틴의 지역 변수 선언 (이 변수는 코루틴이 종료되면 그때 코루틴과 함께 사라진다.)
    while True:
        x = (yield total)   # 코루틴 바깥에서 yield로 값을 받아오고 total 변수의 값을 바깥으로 전달한다.
        total += x

co = sum_coroutine()
output = next(co)     # 코루틴 안의 yield까지 코드를 실행하고 코루틴으로부터 값을 전달 받음: 0
output = co.send(1)   # 코루틴에 숫자 1을 보내고 코루틴으로부터 값을 전달 받음: 0 + 1 = 1
print(output)
output = co.send(2)   # 코루틴에 숫자 2을 보내고 코루틴으로부터 값을 전달 받음: 1 + 2 = 3
print(output)
output = co.send(3)   # 코루틴에 숫자 3을 보내고 코루틴으로부터 값을 전달 받음: 3 + 3 = 6
print(output)

RUN Coroutine
1
3
6


## 제너레이터와 코루틴의 차이점

- 제너레이터는 next 함수(`__next__` 함수)를 반복 호출하여 값을 얻어내는 방식
- 코루틴은 next 함수(`__next__` 함수)를 한 번만 호출한 뒤 send로 값을 주고 받는 방식

# 3. 코루틴을 종료하고 예외 처리하기

---

- 코루틴은 실행 상태를 유지하기 위해 while True: 를 사용해서 끝나지 않는 무한 루프로 동작한다.
- 코루틴을 강제로 종료하고 싶다면 close 함수를 사용한다.

In [15]:
def print_num_coroutine():
    print('RUN Coroutine')
    while True:
        x = (yield)
        print(x, end=' ')

co = print_num_coroutine()
next(co)

for i in range(21):
    co.send(i)

co.close()  # 코루틴 종료

RUN Coroutine
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 

## GeneratorExit 예외 처리하기

- 코루틴 객체에서 close 함수를 호출하면 코루틴이 종료될 때 GeneratorExit 예외가 발생한다.
- 이 예외를 처리하면 코루틴의 종료 시점을 알 수 있다.

In [13]:
def print_num_coroutine():
    print('RUN Coroutine')
    try:
        while True:
            x = (yield)
            print(x, end=' ')
    except GeneratorExit:   # 코루틴이 종료 될 때 GeneratorExit 예외 발생
        print()
        print('CLOSE Coroutine')


co = print_num_coroutine()
next(co)

for i in range(21):
    co.send(i)

co.close()  # 코루틴 종료

RUN Coroutine
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
CLOSE Coroutine


## 코루틴 안에서 예외 발생시키기

- 코루틴 안에 예외를 발생 시킬 때는 throw 함수를 사용한다.
- throw는 말그대로 던지다라는 뜻으로 예외를 코루틴 안으로 던진다.
- throw 함수에 지정한 에러 메시지는 except as의 변수에 들어간다.

아래의 예제 코드는 코루틴에 숫자를 보내서 누적하다가 RuntimeError나 그 외 예외가 발생하면  
에러 메시지를 출력하고 누적된 값을 코루틴 바깥으로 전달하는 코드이다.

In [24]:
def sum_coroutine():
    total = 0
    try:
        while True:
            x = (yield)
            total += x
    except RuntimeError as re:
        print(re)
        yield total
    except BaseException as be:
        print(be)
        yield total

co = sum_coroutine()
next(co)

for i in range(21):
    co.send(i)

output = co.throw(RuntimeError, '예외로 코루틴 끝내기')
print(output)

co = sum_coroutine()
next(co)
output = co.throw(BaseException, '다른 예외 종류로 코루틴 끝내기')
print(output)

예외로 코루틴 끝내기
210
다른 예외 종류로 코루틴 끝내기
0


# 4. 하위 코루틴의 반환값 가져오기

---

- yield from에 코루틴을 지정하면 해당 코루틴에서 return으로 반환한 값을 가져온다.
- yield from은 파이썬 3.3이상부터 사용 가능

아래의 예제 코드는 하위 코루틴에서 숫자를 누적한 뒤 합계를 yield from으로 가져오는 코드이다.

In [59]:
def accumulate():
    print('RUN Child Coroutine')
    total = 0
    try:
        while True:
            x = (yield)             # 부모 코루틴에서 값을 받아옴
            if x is None:           # 받아온 값이 None이면
                print('NONE!!')     
                return total        # 합계 total을 부모 코루틴에게 반환
            else:
                print(f'ADD-{x}', end=" ")
                total += x
    except GeneratorExit:
        print()
        print("###CLOSE Child Coroutine###")
        pass

def sum_coroutine():
    try:
        print('RUN Parent Coroutine')
        while True:
            print('>>> Before Parent call Child')
            total = yield from accumulate()       # 자식(하위) 코루틴에게 바깥에서 받아온 값을 넘겨주고 합계를 가져옴
            print(total)
    except GeneratorExit:
        print("###CLOSE Parent Coroutine###")
        pass

co = sum_coroutine()
next(co)

for i in range(1, 11):
    co.send(i)
co.send(None)

for i in range(1, 11):
    co.send(i)
co.send(None)
co.close()

RUN Parent Coroutine
>>> Before Parent call Child
RUN Child Coroutine
ADD-1 ADD-2 ADD-3 ADD-4 ADD-5 ADD-6 ADD-7 ADD-8 ADD-9 ADD-10 NONE!!
55
>>> Before Parent call Child
RUN Child Coroutine
ADD-1 ADD-2 ADD-3 ADD-4 ADD-5 ADD-6 ADD-7 ADD-8 ADD-9 ADD-10 NONE!!
55
>>> Before Parent call Child
RUN Child Coroutine

###CLOSE Child Coroutine###
###CLOSE Parent Coroutine###



######################  
위의 예제 코드에서 재밌는 점은 하위 코루틴에서 return 키워드로 통해 while문에서 벗어나면서 다시 처음 코드부터 실행된다는 점이다.  
return 키워드에 대해서 공부했다면 당연한 얘기였겠지만 현재 부모 코루틴에서 하위 코루틴의 return 키워드를 통해 값을 받았기 때문에  
하위 코루틴이 메모리에서 사라지졌다가 다시 재생성 된다고 생각했다.  
그래서 실제로 코루틴이 종료될 때의 시점을 알 수 있게 GeneratorExit 예외 처리를 작성 해보았다.  
하지만 예상과는 다르게 종료 예외 처리가 발생되지 않았음을 실행 결과를 통해 알 수 있었다.  
즉, 하위 코루틴이 종료되지 않았음을 알 수 있다.  
이 점을 유의하자.  
######################  

## 그렇다면...

위의 결과를 통해 제너레이터, 코루틴이 종료되고 다시 실행하면 발생하는 예외인 StopIteraion 예외와 연관을 지어보자.  
파이썬 3.6버전 까지는 하위 코루틴에서 값을 반환할 때는 return 대신 raise StopIteration(값) 구문으로 값을 반환했다.  
하지만 StopIteration 예외는 Runtime Exception이다. 즉, 메모리에서 삭제된 후 삭제된 제너레이터 객체를 가동시키려고 할때 발생하는 것이다.  
위의 예제 코드를 통해 알 수 있었듯이 하위 코루틴이 값을 반환할 때에는 메모리에서 삭제되지 않은 것을 알 수 있었고  
즉, 반환할 때 raise StopIteration 구문 사용은 해당 예외 객체의 OOP 5대 원칙인 1원칙, 단일 책임 원칙에 벗어났다고 본다.  
그렇기 때문에 파이썬이 return 키워드를 사용하여 값을 반환하는 것으로 수정했다고 생각된다.